# Parteiprogramme als Datenquelle

In [ ]:
import pandas as pd
import re
import json
import seaborn as sns

from dotenv import load_dotenv
from pathlib import Path
from pdfminer.high_level import extract_text
from studienarbeit.utils.cleaning import Cleaning
from studienarbeit.utils.plots import Plots
from studienarbeit.utils.sentiment import Sentiment
from collections import Counter
from nltk import ngrams
from nltk.tokenize import sent_tokenize
from tqdm import tqdm

In [ ]:
load_dotenv()
sns.set(style="white", palette="muted", rc={"figure.figsize": (20, 8)})
tqdm.pandas()

In [ ]:
data_dir = Path("../../data/party_programs")

In [ ]:
with open("../../data/party_colors.json", "r") as f:
  party_palette = json.load(f)
plots = Plots(party_palette)
sentiment = Sentiment()
cleaning = Cleaning()

## 1. Business Understanding

In unseren Untersuchungszeitraum von 2017 bis 2021, die 19. Wahlperiode des Deutschen Bundestages, fallen neben den Bundestagswahlen 2017 und 2021 die Europawahl 2019 sowie zahlreiche Landtagswahlen, zu denen die antretenden Parteien jeweils ein Wahlprogramm veröffentlichen. Diese Wahlprogramme sind für uns von Interesse, da sie umfangreiche Texte mit der jeweiligen politischen Prägung zu einer großen Bandbreite an Themen bereitstellen und damit viele Informationen dazu bieten, welche Themen von einer Partei immer wieder mit Priorität behandelt werden und welche Sprache dabei verwendet wird. Die Wahlprogramme können üblicherweise als PDF-Datei von der Webseite der jeweiligen Partei bezogen werden.

## 2. Beschaffung der Daten

In Summe stehen 82 Wahlprogramme von AfD, FDP, Grünen, Linken, SPD und CDU/CSU zur Verfügung.

In [ ]:
df_party_programs_overview = pd.read_csv(data_dir / "overview.csv", delimiter=";")
df_party_programs_overview

### Hinweis: Nicht parsbare Wahlprogramme

Der entwickelte Algorithmus zum Auslesen des Texten und parsen der einzelnen Paragraphen eines Wahlprogramms funktioniert für die meisten gegebenen Wahlprogramme im PDF-Format. Es gibt jedoch einige Ausnahmen, bei denen entweder der Text nicht erkannt werden kann oder die Formatierung so ist, dass der Algorithmus ohne erhebliche Anpassungen nicht funktioniert. Dies betrifft die folgenden Wahlprogramme:
- AfD + FDP LTW MV 21
- FDP + CDU LTW 21 Berlin
- AfD LTW 21 Sachsen-Anhalt
- Grüne LTW 21 Bade-Württemberg
- SPD LTW 21 Rheinland-Pfalz
- SPD + CDU LTW 20 Hamburg
- AfD LTW 19 Thüringen
- FDP LTW 19 Brandenburg
- AfD Europawahl 2019
- AfD + SPD LTW 18 Hessen
- AfD + Linke LTW 18 Bayern

In [ ]:
def remove_phrases_from_file(text, party):
    df_phrases = pd.read_csv(data_dir / "src" / party / "phrases.csv", delimiter=";", keep_default_na=False)
    for _, row in df_phrases.iterrows():
        text = re.sub(row["phrase"], row["new"], text, flags=re.IGNORECASE)
    return text

def initial_cleaning(text, lower=False, gender_symbols=["*", ":"]):
    if lower:
        text = text.lower()
    
    text = text.replace("-\n", "- ")
    text = re.sub("-\s+", "-", text)
    text = text.replace("-oder", "- oder")
    text = text.replace("-und", "- und")
    text = re.sub("([a-zßäöü])-([a-zßäöü])", r"\1\2", text)
    text = re.sub("[\u2022\u2023\u25E6\u2043\u2219\uf0b7\u25fc]\s", "", text)
    text = text.replace("\n", " ")
    text = re.sub("\s+", " ", text)
    text = text.strip()
    
    text = cleaning.clean_gender(text, gender_symbols)
        
    # Party specific cleaning
    for party in ["AfD", "FDP", "Grüne", "Linke", "SPD", "Union"]:
        text = remove_phrases_from_file(text, party)
    
    # Remove possible newly occured double spaces
    text = re.sub("\s+", " ", text)
    
    return text.strip()

In [ ]:
def merge_paragraphs(all_paragraphs):
    tmp = []
    for p in all_paragraphs:        
        if (p[-1] in [".", "!", "?"]):
            if ((len(tmp) == 0) and (p[0].islower())):
                continue
            tmp.append(p)
            yield " ".join(tmp)
            tmp = []
        elif (p[-1] == ":"): 
            continue
        elif ((p[0].isupper()) and (tmp == [])):
            tmp.append(p)

In [ ]:
df_all_paragraphs = pd.DataFrame(columns=["text_orig", "party"])

for _, program in df_party_programs_overview.iterrows():
    all_text = extract_text(data_dir / "src" / program["party"] / f"{program['election']}.pdf", page_numbers=range(program["first_page"] - 1, program["last_page"]))

    all_paragraphs = list(filter(lambda x : 150 <= len(x), all_text.split("\n\n")))
        
    all_paragraphs = [initial_cleaning(x) for x in all_paragraphs]
    all_paragraphs = list(filter(lambda x : 150 <= len(x), all_paragraphs))
    all_paragraphs = merge_paragraphs(all_paragraphs)

    df_current_party = pd.DataFrame(all_paragraphs, columns=["text_orig"])
    df_current_party["party"] = program["party"]
    df_current_party["election_type"] = program["election"][:3]
    df_current_party["election"] = program["election"]

    df_all_paragraphs = pd.concat([df_all_paragraphs, df_current_party])

In [ ]:
df_all_paragraphs

## 3. Data Understanding

In [ ]:
df_understanding = df_all_paragraphs.copy().reset_index(drop=True)

`Anzahl an Paragraphen pro Partei`

In [ ]:
plots.party_count(df_understanding, title="Anzahl an Paragraphen nach Partei")

### `Anzahl an Paragraphen nach Art der Wahl`

In [ ]:
plots.grouped_party_count(df_understanding, "election_type", "Anzahl an Paragraphen nach Art der Wahl")

In [ ]:
df_understanding.groupby("party").count()

### `Häufigste 3-grams`

In [ ]:
for party in ["AfD", "FDP", "Grüne", "Linke", "SPD", "Union"]:
    df_current_party = df_understanding[df_understanding["party"] == party]
    ngram_counts = Counter(ngrams(" ".join(df_current_party["text_orig"].tolist()).split(), 3))
    print(f"{party}: " + str(ngram_counts.most_common(5)))
    print("\n")

### `Verteilung der Anzahl an...`

In [ ]:
df_understanding["char_count"] = df_understanding["text_orig"].progress_apply(len).astype("int16")
df_understanding["word_count"] = df_understanding["text_orig"].progress_apply(lambda x : len(x.split())).astype("int16")
df_understanding["sentence_count"] = df_understanding["text_orig"].progress_apply(lambda x : len(sent_tokenize(x))).astype("int16")

#### `Zeichen`

In [ ]:
plots.text_count(df_understanding, "char_count", "Anzahl an Zeichen in Paragraphen nach Partei", "Anzahl Zeichen", 2500)

#### `Wörtern`

In [ ]:
plots.text_count(df_understanding, "word_count", "Anzahl an Wörtern in Paragraphen nach Partei", "Anzahl Wörter",300)

In [ ]:
df_understanding

In [ ]:
df_understanding.sort_values(by="word_count", ascending=False).head(10)

#### `Sätzen`

In [ ]:
plots.text_count(df_understanding, "sentence_count", "Anzahl an Sätzen in Paragraphen nach Partei", "Anzahl Sätze", 15)

## 4. Data Preparation

In [ ]:
df_prep = df_all_paragraphs.copy().reset_index(drop=True)

In [ ]:
df_prep["clean_text"] = df_prep["text_orig"].progress_apply(lambda x: cleaning.clean_text(x, keep_punctuation=True, keep_upper=True)).astype("string[pyarrow]")
df_prep["tokenized_text"] = df_prep["clean_text"].progress_apply(lambda x: cleaning.filter_text(cleaning.stemm_text(x))).astype("string[pyarrow]")

df_prep

In [ ]:
df_prep.groupby("party").count()

In [ ]:
df_prep.duplicated(subset=["clean_text"]).sum()

In [ ]:
df_prep = df_prep.drop_duplicates(subset=["clean_text"]).reset_index(drop=True)

In [ ]:
df_prep.duplicated(subset=["clean_text"]).sum()

In [ ]:
df_final = df_prep.copy().reset_index(drop=True).drop(columns=["text_orig", "election_type", "election"])

In [ ]:
with open("../../data/party_encoding.json", "r") as f:
  party_encoding = json.load(f)
  df_final["party"] = df_final["party"].map(party_encoding)

In [ ]:
df_final

In [ ]:
df_final.to_parquet(data_dir / "party_programs.parquet", index=False)